In [7]:
import pandas as pd
import os
import re
import numpy as np
import pprint
import datetime

In [8]:
df = pd.read_excel("../data/fomc_dissents_data.xlsx",skiprows=3)

In [9]:
df["Date"] = df["FOMC Meeting"].apply(lambda x:str(x).split(" ")[0])
df['FOMC Votes'] = df['FOMC Votes'].apply(lambda x:0 if np.isnan(x) else x)
print(df.columns)

Index(['Year', 'FOMC Meeting', 'Chair', 'Dissent (Y or N)', 'FOMC Votes',
       'Votes for Action', 'Votes Against Action',
       'Number Governors Dissenting', 'Number Presidents Dissenting',
       'No. Governors for Tighter', 'No. Governors for Easier',
       'No. Presidents for Tighter', 'No. Presidents for Easier',
       'Dissenters Tighter', 'Dissenters Easier',
       'Dissenters Other/Indeterminate', 'Date'],
      dtype='object')


In [11]:
df['date'] = pd.to_datetime(df["Date"])
df['start_date'] = df['date'] - pd.Timedelta('1 days')
df['start_date']=df['start_date'].dt.date
df['date']=df['date'].dt.date
df[['date','start_date']].head()

,date,start_date
0,1936-03-19,1936-03-18
1,1936-05-25,1936-05-24
2,1936-11-20,1936-11-19
3,1937-01-26,1937-01-25
4,1937-03-15,1937-03-14


In [12]:

voter_df = pd.DataFrame()

for index,row in df.iterrows():
    voters = []
    num_voters = int(row['FOMC Votes'])
    date_path = '../../../collection/python/data/transcript_raw_text/{}.txt'.format(row['date'])
    if not os.path.exists(date_path):
        print("Date not found")
        date_path = '../../../collection/python/data/transcript_raw_text/{}.txt'.format(row['start_date'])
        if not os.path.exists(date_path):
            print("Alternative date not found")
            continue
        else:
            print('Process alternative date')
    with open(date_path) as f:
        broken = False
        broken_starts = 0
        broken_ends = 0
        lines = f.readlines()
        
        '''First Check For Broken Title'''

        print("CHECKING USING FRAGMENT HEURISTIC")
        for line in lines[:200]:
            if line.strip():
                if broken_ends==0:
                    title_frag = re.match(r'^(?:PRESENT: |PRESENT. )?(?:Mr.|Ms.|Mt. )$',line.strip())
                    if title_frag:
                        if not broken:
                            broken = True
                        print("Broken Begining")
                        print(title_frag.group(0))
                        title_frag_string = str(title_frag.group(0)).replace("PRESENT: ","")
                        voters.append(title_frag_string)
                        broken_starts+=1
                        continue
                if broken and broken_ends<len(voters):
                    name_fragment = re.match('^[A-Z][a-z][A-Za-z]*',line.strip())
                    if name_fragment:
                        print(name_fragment)
                        if broken_ends==0:
                            print("Found {} Fragments".format(len(voters)))
                            print(voters)
                        print("Broken Ending")
                        print(name_fragment.group(0))
                        voters[broken_ends] = voters[broken_ends]+" "+str(name_fragment.group(0))
                        broken_ends+=1
        
        '''Check using Mr. Regex'''
        
        if len(voters)==0:
            print("CHECKING TITLE REGEX")
            for line in lines[:200]:
                '''Then check for valid input'''
                voter_line = re.findall(r'(?:Mr.|Ms.) [A-Z][a-zA-Z]*',line.strip())
                if voter_line:
                    #print(voter_line)
                    voters.append(voter_line[0])
                    if len(voters)>=num_voters:
                        break
        
        '''Check Last Name Regex'''
        
        if len(voters) == 0:
            print("Checking POST-PRESENT-NAME HEURISTIC")
            found_present = False
            for line in lines[:200]:
                if "PRESENT:" in line.strip() or "PRESENT." in line.strip():
                    found_present = True
                    present_line = line.split(",")[0].strip().replace("PRESENT","")
                    name_text = re.match('[A-Z][a-z]*\s?(?:[A-Z][a-z]*)?',present_line)
                    if name_text:
                        voters.append(name_text.group(0))
                    continue
                if found_present:
                    print(line)
                    name_text = re.match('[A-Z][a-z]*\s?(?:[A-Z][a-z]*)?',line.split(",")[0].strip())
                    if name_text:
                        voters.append(name_text.group(0))
                    if len(voters)>=num_voters:
                        break
    print('Date:{}'.format(row['Date']))
    print("Broken Status:{}".format(broken))
    print("Voter Number:{}".format(num_voters))
    print("Voters Found:{}".format(len(voters)))
    pprint.pprint(voters)
    voter_df = voter_df.append({
        "Date":row['FOMC Meeting'],
        "voters_expected":num_voters,
        "voters_observed":len(voters),
        "Voters":voters if num_voters==len(voters) else None,
    },ignore_index=True)
    print("="*50)
print(voter_df)

Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date not found
Date not found
Alternative date no

CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 5), match='Burns'>
Found 12 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Burns
<_sre.SRE_Match object; span=(0, 7), match='Volcker'>
Broken Ending
Volcker
<_sre.SRE_Match object; span=(0, 5), match='Black'>
Broken Ending
Black
<_sre.SRE_Match object; span=(0, 8), match='Coldwell'>
Broken Ending
Coldwell
<_sre.SRE_Match object; span=(0, 7), match='Gardner'>
Broken Ending
Gardner
<_sre.SRE_Match object; span=(0, 7), match='Jackson'>
Broken Ending
Jackson
<_sre.SRE_Match object; span=(0, 7), match='Kimbrel'>
Broken Ending
Kimbrel
<_sre.SRE_Match object; span=(0, 5), match='Lilly'>
Broken Ending
Lilly
<_sre.SRE_Match

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1978-03-10
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Burns',
 'Mr. Coldwell',
 'Mr. Eastburn',
 'Mr. Jackson',
 'Mr. Miller',
 'Mr. Wallich',
 'Mr. Willes',
 'Mr. Winn',
 'Mr. Volcker',
 'Mr. Kimbrel']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 6), match='Miller'>
Found 10 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Miller
<_sre.SRE_Match object; span=(0, 7), match='Volcker'>
Broken Ending
Volcker
<_sre.SRE_Match object; span=(0, 8), match='Baughman'>
Broken Ending
Baughman
<_sre.SRE_Match object; span=(0, 8), match='Coldwell'>
Broken Ending
Coldwell
<_sre.SRE_Match object; span=(0, 8), match='Eastburn'>
Broken Ending
Eastburn
<_sre

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1980-05-06
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Volcker',
 'Mr. Guffey',
 'Mr. Morris',
 'Mr. Rice',
 'Mr. Roos',
 'Mr. Schultz',
 'Mr. Solomon',
 'Mr. Wallich',
 'Mr. Altmann',
 'Mr. Bernard']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 7), match='Altmann'>
Found 7 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Altmann
<_sre.SRE_Match object; span=(0, 7), match='Bernard'>
Broken Ending
Bernard
<_sre.SRE_Match object; span=(0, 8), match='Petersen'>
Broken Ending
Petersen
<_sre.SRE_Match object; span=(0, 6), match='Oltman'>
Broken Ending
Oltman
<_sre.SRE_Match object; span=(0, 7), match='Mannion'>
Broken Ending
Mannion
<_sre.SRE_Match object; span=(0, 7), match='Axilrod'>
Broken Ending
Axilrod
<_sre.SRE_Match object; spa

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1982-12-21
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Volcker',
 'Mr. Solomon',
 'Mr. Balles',
 'Mr. Black',
 'Mr. Ford',
 'Mr. Gramley',
 'Mr. Martin',
 'Mr. Partee',
 'Mr. Rice',
 'Mr. Wallich',
 'Mr. Axilrod',
 'Mr. Altmann']
Date not found
Process alternative date
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1983-02-09
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Volcker',
 'Mr. Solomon',
 'Mr. Balles',
 'Mr. Black',
 'Mr. Ford',
 'Mr. Gramley',
 'Mr. Martin',
 'Mr. Partee',
 'Mr. Rice',
 'Mr. Wallich',
 'Mr. Axilrod',
 'Mr. Bernard']
Date not found
Process alternative date
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1983-03-29
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Volcker',
 'Mr. Solomon',
 'Mr. Gramley',
 'Mr. Guffey',
 'Mr. Keehn',
 'Mr. Martin',
 'Mr. Morris',
 'Mr. Partee',
 'Mr. Rice',
 'Mr. Roberts',
 'Mr. Wallich',
 'Mr. Axilrod']
CHECKI

...

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1994-09-27
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Blinder',
 'Mr. Broaddus',
 'Mr. Forrestal',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. LaWare',
 'Mr. Lindsey',
 'Mr. Parry',
 'Ms. Phillips',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1994-11-15
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Blinder',
 'Mr. Broaddus',
 'Mr. Forrestal',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. LaWare',
 'Mr. Lindsey',
 'Mr. Parry',
 'Ms. Phillips',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 4), match='Kohn'>
Found 8 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Kohn
<_sre.SRE_Match object; spa

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1997-09-30
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Broaddus',
 'Mr. Guynn',
 'Mr. Kelley',
 'Mr. Moskow',
 'Mr. Meyer',
 'Mr. Parry',
 'Ms. Phillips',
 'Ms. Rivlin']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1997-11-12
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Broaddus',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Guynn',
 'Mr. Kelley',
 'Mr. Meyer',
 'Mr. Moskow',
 'Mr. Parry',
 'Ms. Phillips',
 'Ms. Rivlin']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1997-12-16
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Broaddus',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Guynn',
 'Mr. Kelley',
 'Mr. Moskow',
 'Mr. Meyer',
 'Mr. Parry',
 'Ms. Phillips',
 'Ms. Rivlin']
Date not found
Process alternative date
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REG

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-01-31
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kelley',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-03-20
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kelley',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-04-18
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kelley',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-05-15
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonou

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2004-01-28
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. Geithner',
 'Mr. Bernanke',
 'Ms. Bies',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Ms. Minehan',
 'Mr. Olson',
 'Ms. Pianalto',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2004-03-16
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. Geithner',
 'Mr. Bernanke',
 'Ms. Bies',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Ms. Minehan',
 'Mr. Olson',
 'Ms. Pianalto',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2004-05-04
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. Geithner',
 'Mr. Bernanke',
 'Ms. Bies',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Ms. Minehan',
 'Mr. Olson',
 'Ms. Pianalto',
 'Mr. Poole']
Date not found
Process alternative date
CHECKING USING FRAGMENT HEU

Date not found
Process alternative date
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2007-03-21
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. Geithner',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Ms. Minehan',
 'Mr. Mishkin',
 'Mr. Moskow',
 'Mr. Poole',
 'Mr. Warsh']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2007-05-09
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. Geithner',
 'Mr. Hoenig',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Ms. Minehan',
 'Mr. Mishkin',
 'Mr. Moskow',
 'Mr. Poole',
 'Mr. Warsh']
Date not found
Process alternative date
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2007-06-28
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. Geithner',
 'Mr. Hoeing',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Ms. Minehan',
 'Mr. Mishkin',
 'Mr. Moskow',
 'Mr. Poole',
 'Mr. Warsh']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2007-08-07
Broken Status:Fals

In [14]:
#print(len(voter_df[voter_df["Voters"].isna()]))
voter_df[voter_df["Voters"].isna()].reset_index()

,index,Date,Voters,voters_expected,voters_observed
0,26,1978-05-05,None,10.0,4.0
1,36,1979-03-20,None,10.0,8.0
2,41,1979-09-18,None,12.0,6.0
3,42,1979-10-06,None,12.0,9.0
4,46,1980-03-07,None,11.0,5.0
5,50,1980-05-20,None,11.0,7.0
6,51,1980-07-09,None,12.0,7.0
7,53,1980-09-16,None,12.0,6.0
8,55,1980-11-18,None,12.0,7.0
9,56,1980-11-26,None,11.0,5.0


In [15]:
voter_df[voter_df.voters_observed==0]

,Date,Voters,voters_expected,voters_observed
300,2010-01-27,None,10.0,0.0
301,2010-03-16,None,10.0,0.0
302,2010-04-28,None,10.0,0.0
303,2010-06-23,None,10.0,0.0
304,2010-08-10,None,10.0,0.0
305,2010-09-21,None,9.0,0.0
306,2010-11-03,None,11.0,0.0
307,2010-12-14,None,11.0,0.0
308,2011-01-26,None,11.0,0.0
309,2011-03-15,None,10.0,0.0
